In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import os

In [36]:
degs = pd.read_csv('/lustre/scratch125/casm/team215mg/pg21_rotation/G12D_DEG_list.csv')
reference_adata = sc.read_h5ad('/lustre/scratch125/casm/team215mg/pg21_rotation/Coefficients_CellDrift/selection_rep_1/CellDrift_object_time_24.h5ad')


# define list of cell states, treatments and genes
cell_states = np.unique(reference_adata.obs['final_group_labeled'])
treatments = np.unique(reference_adata.obs['drug'])
treatments = np.delete(treatments, [0])
gene_list = degs.drop_duplicates(subset = 'gene')['gene']

In [67]:
# identify significant overlapping DEGs (ie. identified as DEGs before with pseudobulk and now with CellDrift), then save 

timepoints = [24, 48, 72]

for replicate in range(1, 4):
    for time in timepoints:
        coefficients = pd.read_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Coefficients_CellDrift/selection_rep_{replicate}/Contrast_Coefficients_time_{time}.txt', sep = '\t')
        for state in cell_states:
            for drug in treatments:
                # identify all significant DEGs per replicate/time/state/condition
                significant_degs = coefficients[coefficients['contrast'] == f'{state}_{drug}-{state}_DMSO'].loc[coefficients['p_fdr'] < 0.05, ['gene']]
                print(state, drug, len(significant_degs))
                # identify and verify overlapping DEGs between pseudobulk DEGs and CellDrift DEGs
                genes_and_clusters = pd.read_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Temporal_CellDrift/{state}/{drug}/FDA_clusters.txt', sep = '\t')
                cluster_counts = overlap['clusters_fuzzy'].value_counts()
                cluster_counts.to_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Temporal_CellDrift/{state}/{drug}/cluster_counts.tsv', sep = '\t')
                overlap_sig = gene_list[gene_list.isin(significant_degs['gene']) == True]
                print(len(overlap_sig))
                overlap = genes_and_clusters.loc[genes_and_clusters['genes'].isin(overlap_sig) == True, ['genes', 'clusters_fuzzy']]
                overlap.to_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Temporal_CellDrift/{state}/{drug}/overlap_deg_clusters.tsv', sep = '\t')
                cluster_counts = overlap['clusters_fuzzy'].value_counts()
                cluster_counts.to_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Temporal_CellDrift/{state}/{drug}/cluster_counts.tsv', sep = '\t')
                print(len(overlap))
                

92
Cycling MRTX1133 41
41
118
Cycling MRTX1133_SHP099 60
60
88
Cycling MRTX1133_afatinib 47
47
28
Cycling SHP099 10
10
45
Cycling afatinib 20
20
99
Fetal MRTX1133 22
22
153
Fetal MRTX1133_SHP099 64
64
200
Fetal MRTX1133_afatinib 81
81
42
Fetal SHP099 12
12
78
Fetal afatinib 24
24
58
Respiration MRTX1133 16
16
82
Respiration MRTX1133_SHP099 30
30
100
Respiration MRTX1133_afatinib 36
36
18
Respiration SHP099 3
3
30
Respiration afatinib 12
12
149
Secretory MRTX1133 93
93
181
Secretory MRTX1133_SHP099 106
106
146
Secretory MRTX1133_afatinib 84
84
73
Secretory SHP099 38
38
127
Secretory afatinib 71
71
146
Stem MRTX1133 51
51
198
Stem MRTX1133_SHP099 88
88
230
Stem MRTX1133_afatinib 102
102
117
Stem SHP099 25
25
61
Stem afatinib 23
23
67
Stress MRTX1133 41
41
154
Stress MRTX1133_SHP099 68
68
181
Stress MRTX1133_afatinib 90
90
85
Stress SHP099 27
27
184
Stress afatinib 71
71
108
Cycling MRTX1133 41
41
156
Cycling MRTX1133_SHP099 53
53
154
Cycling MRTX1133_afatinib 67
67
28
Cycling SHP099 12
1

In [63]:
coefficients = pd.read_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Coefficients_CellDrift/selection_rep_1/Contrast_Coefficients_time_24.txt', sep = '\t')

significant_degs = coefficients[coefficients['contrast'] == f'Cycling_MRTX1133-Cycling_DMSO'].loc[coefficients['p_fdr'] < 0.05, ['gene']]

genes_and_clusters = pd.read_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Temporal_CellDrift/Cycling/MRTX1133/FDA_clusters.txt', sep = '\t')

In [64]:
overlap = genes_and_clusters.loc[gene_list.isin(significant_degs['gene']) == True, ['genes', 'clusters_fuzzy']]
overlap

,genes,clusters_fuzzy
226,COL17A1,0
227,COL8A1,17
263,DKK1,2
267,DMBT1,5
273,DST,18
298,FAM155A,7
303,FDPS,9
338,GNAI1,5
427,KRT20,2
448,LINC01811,0


In [65]:
overlap_sig = gene_list[gene_list.isin(significant_degs['gene']) == True]
overlap_sig

226        AMACR
227         AMOT
263        ATAD2
267       ATP10B
273        ATXN1
298        BNIP5
303        BRIP1
338       CAB39L
427         CFTR
448        CLSPN
511       DIAPH3
527          DTL
528         DTNA
554      EIF2AK3
626        FGF20
642       GABBR1
716     HIST1H3B
726     HIST1H4C
763      IGFBPL1
774        ITGB6
949        MCTP1
997        MYH14
1126     PLA2G2A
1141      PMFBP1
1234        RGS6
1259      SAMD4A
1276      SEMA5A
1317      SLC7A8
1319     SLCO3A1
1341      SORBS2
1368      STK38L
1385       SYTL2
1443    TNFRSF19
1446        TNS4
1485      TUBB4B
1538      ZBTB20
1562      ZNF704
1901       LSAMP
2520        XACT
2954       TIMP3
3157       EPHB6
Name: gene, dtype: object

In [66]:
genes_and_clusters[genes_and_clusters['genes'].isin(overlap_sig) == True].count()

Unnamed: 0         41
genes              41
clusters_kmeans    41
clusters_fuzzy     41
dtype: int64

In [50]:
genes_and_clusters.isin(overlap_sig == True)

,Unnamed: 0,genes,clusters_kmeans,clusters_fuzzy
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
810,False,False,False,False
811,False,False,False,False
812,False,False,False,False
813,False,False,False,False


In [55]:
celldrift_genes = genes_and_clusters['genes']